In [2]:
import os
import cv2
import zipfile
import random
import tensorflow as tf
import shutil
import numpy as np
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [3]:
def split_and_copy_files(source_subdir, train_subdir, validation_subdir, test_size=0.2):
    files = [f for f in os.listdir(source_subdir) if os.path.isfile(os.path.join(source_subdir, f))]
    train_files, validation_files = train_test_split(files, test_size=test_size)
    
    for file in train_files:
        shutil.copy(os.path.join(source_subdir, file), os.path.join(train_subdir, file))
    
    for file in validation_files:
        shutil.copy(os.path.join(source_subdir, file), os.path.join(validation_subdir, file))

dataset_dir = "C:/Users/Admin/Desktop/Uni/GIN515/dataset/"
train_dir = "C:/Users/Admin/Desktop/Uni/GIN515/training/"
validation_dir = "C:/Users/Admin/Desktop/Uni/GIN515/validation/"

# Directories for 'with_mask' and 'without_mask'
subdirs = ['with_mask', 'without_mask']

# Ensure the main and subdirectories exist
for subdir in subdirs:
    os.makedirs(os.path.join(train_dir, subdir), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, subdir), exist_ok=True)

# Process each subdirectory
for subdir in subdirs:
    source_subdir = os.path.join(dataset_dir, subdir)
    train_subdir = os.path.join(train_dir, subdir)
    validation_subdir = os.path.join(validation_dir, subdir)

    split_and_copy_files(source_subdir, train_subdir, validation_subdir)

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu',
                           input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
        tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])


train_datagen = ImageDataGenerator(rescale=1.0/255.)
validation_datagen = ImageDataGenerator(rescale=1.0/255.)


train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=16,
                                                    class_mode='binary',
                                                    target_size=(128, 128))

validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                    batch_size=16,
                                                    class_mode='binary',
                                                    target_size=(128, 128))


history = model.fit(train_generator, epochs=5, validation_data= validation_generator)

model.save("my_model.h5")

Found 3924 images belonging to 2 classes.
Found 2650 images belonging to 2 classes.
Epoch 1/5
 34/246 [===>..........................] - ETA: 39s - loss: 0.5775 - acc: 0.7096

C:\Users\Admin\anaconda3\lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


246/246 [==============================] - 71s 286ms/step - loss: 0.2995 - acc: 0.8756 - val_loss: 0.2144 - val_acc: 0.9279
Epoch 2/5
246/246 [==============================] - 50s 202ms/step - loss: 0.1570 - acc: 0.9422 - val_loss: 0.0976 - val_acc: 0.9626
Epoch 3/5
246/246 [==============================] - 47s 189ms/step - loss: 0.0976 - acc: 0.9661 - val_loss: 0.0637 - val_acc: 0.9770
Epoch 4/5
246/246 [==============================] - 46s 186ms/step - loss: 0.0468 - acc: 0.9839 - val_loss: 0.0264 - val_acc: 0.9925
Epoch 5/5
246/246 [==============================] - 44s 180ms/step - loss: 0.0361 - acc: 0.9862 - val_loss: 0.0387 - val_acc: 0.9849


In [10]:
import cv2
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('my_model.h5')

# Start video capture
cap = cv2.VideoCapture(0)

# Load the Haar Cascade for face detection
cascade_path = r"C:\Users\Admin\Downloads\haarcascade_frontalface_default.xml"
face_cascade = cv2.CascadeClassifier(cascade_path)

while True:
    ret, frame = cap.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (128, 128))  # Resize to the expected input size of the model
        face_roi = face_roi / 255.0  # Normalize pixel values if needed
        face_roi = np.expand_dims(face_roi, axis=0)  # Add the batch dimension

        prediction = model.predict(face_roi)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        threshold = 0.4  
        if prediction > threshold:
            cv2.putText(frame, 'Mask', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        else:
            cv2.putText(frame, 'No Mask', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 44ms/step
